In [4]:
!pip install gensim

     |████████████████████████████████| 24.2MB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 122kB 45.5MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-cp36-none-any.whl size=107098 sha256=24cd87688072dfa7a97b938e26cff75cb02f6378024f8c96d4d8b72e8fde00da
  Stored in directory: /home/ubuntu/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open


In [1]:
import re
import os
import nltk
import io
import math
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tqdm import tqdm
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models.keyedvectors import KeyedVectors

DATA_DIR="data/"
WORD2VEC_BIN_PATH=DATA_DIR+"SO_vectors_200.bin"
ANSWERS_CSV_PATH=DATA_DIR+"Answers.csv"
QUESTIONS_CSV_PATH=DATA_DIR+"Questions.csv"
TAGS_CSV_PATH=DATA_DIR+"Tags.csv"

In [2]:
word_vect = KeyedVectors.load_word2vec_format(WORD2VEC_BIN_PATH, binary=True)

In [3]:
idToTagIndex = {}          #dict mapping post ID to a list of tag indices
tagToTagIndex = {}         #dict mapping tag to tag index

In [4]:
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [5]:
questions_data = None
with io.open(QUESTIONS_CSV_PATH, 'r',encoding='utf-8',errors='ignore') as question_input:
    questions_data = pd.read_csv(question_input, engine='python')

print(questions_data.columns)
questions_data = questions_data[['Id', 'Body']]
questions_data.insert(len(questions_data.columns), 'Code', "")

a = re.compile(r'<pre><code>([^<]*)</code></pre>')
b = re.compile(r'<.*?>')
questions_data['Code'] = questions_data['Body'].apply(lambda x: ' '.join(re.findall(a, x)))

def clean(text):
    x = re.sub(a, '', text)
    x = re.sub(b, '', x)
    x = x.replace('\n\n', '\n')
    return x
questions_data['Body'] = questions_data['Body'].apply(clean)
    
#     questions_data['Body'] = questions_data['Body'].apply(nltk.tokenize.word_tokenize) #need to fix: don't convert
#     # C#, C++ to C
#     questions_data['Body'] = questions_data['Body'].apply(lambda x: [word for word in x if word.isalnum()])
#     questions_data['Body'] = questions_data['Body'].apply(lambda x: [word.lower() for word in x])

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')


In [6]:
from collections import defaultdict

idToTagIndex = {}          #dict mapping post ID to a list of tag indices
tagToTagIndex = {}         #dict mapping tag to tag index
tagIndexToTag = {}
tagToFrequency = defaultdict(lambda: 0)

tag_data = None
with open(TAGS_CSV_PATH) as tag_input:
    tag_data = pd.read_csv(tag_input)
tagIndex = 0
for index, row in tqdm(tag_data.iterrows()):
    currId = int(row[0])
    currTag = row[1]
    if currTag not in tagToTagIndex:
        tagToTagIndex[currTag] = tagIndex
        tagIndexToTag[tagIndex] = currTag
        currTagIndex = tagIndex
        tagIndex += 1

    else:
        currTagIndex = tagToTagIndex[currTag]  

    tagToFrequency[currTagIndex] += 1        

    if currId not in idToTagIndex.keys():
        idToTagIndex[currId] = [tagToTagIndex[row[1]]]
    else:
        idToTagIndex[currId].append(tagToTagIndex[row[1]])
            

3750994it [10:14, 6106.65it/s]


In [8]:
print("Number of examples: ", len(idToTagIndex))
print(questions_data.loc[1])
print(len(questions_data.index))

Number of examples:  1264216
Id                                                     90
Body    Are there any really good tutorials explaining...
Code                                                     
Name: 1, dtype: object
1264216


In [10]:
# find 10 most common tags

tagToFrequencyList = []

for key, value in tagToFrequency.items():
    temp = [key, value]
    tagToFrequencyList.append(temp)
    
tagToFrequencyList.sort(reverse=True, key=lambda x: x[1])

for tag in tagToFrequencyList[:10]:
    print(f"{tagIndexToTag[tag[0]]} ({tag[0]}): {tag[1]} times")
    
mostCommonTags = {}
for counter, tag in enumerate(tagToFrequencyList[:10]):   #currently takes top 10 tags
    mostCommonTags[tag[0]] = counter


javascript (132): 124155 times
java (89): 115212 times
c# (14): 101186 times
php (76): 98808 times
android (395): 90659 times
jquery (370): 78542 times
python (196): 64601 times
html (58): 58976 times
c++ (18): 47591 times
ios (2045): 47009 times


In [11]:
# list(idToTagIndex.values())[:10]
print(mostCommonTags)

{132: 0, 89: 1, 14: 2, 76: 3, 395: 4, 370: 5, 196: 6, 58: 7, 18: 8, 2045: 9}


In [41]:
idToTenTags = {}

for postId, tags in idToTagIndex.items():
    containsTopTenTags = [mostCommonTags[tag] for tag in tags if tag in mostCommonTags.keys()]
    idToTenTags[postId] = containsTopTenTags
    
questions_data['Top-Tags'] = questions_data['Id'].apply(lambda x: idToTenTags[x])

mostCommonTagsStrToTagRank={} #dict of most common tag string to tag rank
for tagIdx, tagRank in mostCommonTags.items():
    mostCommonTagsStrToTagRank[tagIndexToTag[tagIdx]] = tagRank
print(mostCommonTagsStrToTagRank)

mostCommonTagsTagRankToStr = {v:k for k, v in mostCommonTagsStrToTagRank.items()}
print(mostCommonTagsTagRankToStr)

'''
for index,row in questions_data.iterrows():
    if len(row['Top-Tags']) !=0:
        k = row['Top-Tags']
        row['Top-Tags'] = [mostCommonTagsTagRankToStr[k] for  k in row['Top-Tags']]
        print(row)
        row['Top-Tags'] = k
        break
'''

print(questions_data.loc[0])

{'javascript': 0, 'java': 1, 'c#': 2, 'php': 3, 'android': 4, 'jquery': 5, 'python': 6, 'html': 7, 'c++': 8, 'ios': 9}
{0: 'javascript', 1: 'java', 2: 'c#', 3: 'php', 4: 'android', 5: 'jquery', 6: 'python', 7: 'html', 8: 'c++', 9: 'ios'}
RangeIndex(start=0, stop=1264216, step=1)


In [166]:
dedup_q_data = pd.DataFrame(questions_data)
for index,row in tqdm(questions_data.iterrows()):
    if len(questions_data['Top-Tags'][index]) == 0:
        dedup_q_data = dedup_q_data.drop([index])
print(len(dedup_q_data.index))

30417it [1:17:27,  6.72it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

38757it [1:37:56,  6.59it/s]


KeyboardInterrupt: 

In [174]:
questions_data_20_list = []
questions_data_20= questions_data.head(n=5)
questions_data_300k= questions_data.head(n=300000)
print(len(questions_data_100k.index))

qd = pd.DataFrame(["this is good", "this is nice"], columns=['Body'])
print(qd['Body'].values.tolist())

100000
['this is good', 'this is nice']


In [110]:
text_dataset = tf.data.Dataset.from_tensor_slices(["foo", "bar", "baz"])
max_features = 5000
max_len = 4 
embedding_dims = 2

vectorize_layer = TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

print(type(text_dataset.batch(64)))
vectorize_layer.adapt(text_dataset.batch(64))


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

input_data = [["foo qux bar"], ["qux baz"]]
print(model.predict(input_data))
print(vectorize_layer.get_vocabulary()[:5])

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
[[2 1 4 0]
 [1 3 0 0]]
[b'foo', b'baz', b'bar']


In [175]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(questions_data_100k['Body'].values.tolist())
vectorizer.adapt(text_ds.batch(200))
vectorizer.get_vocabulary()[:10]

[b'the', b'to', b'i', b'a', b'is', b'and', b'in', b'of', b'it', b'this']

In [112]:
# Create the model that uses the vectorize text layer
model = tf.keras.models.Sequential()
print(vectorizer.get_vocabulary()[3])
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer)
input_data = [["I've written a database generation script in"]]
model.predict(input_data)

b'a'


array([[  73,  480,    5,  128, 2658,  169,    8,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [176]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

questions_X = vectorizer(np.array([[s] for s in questions_data_100k['Body'].values])).numpy()
questions_y = mlb.fit_transform(np.array(questions_data_100k['Top-Tags'].values))
print(questions_X)
print(questions_data_100k['Top-Tags'].values[5])

train_X, test_X, train_y, test_y = train_test_split(questions_X, questions_y, train_size=0.75, random_state=200)

[[ 66 439   5 ...   0   0   0]
 [ 36  33  38 ...   0   0   0]
 [ 59 119 231 ...   0   0   0]
 ...
 [  4  35 912 ...   0   0   0]
 [  4  52   3 ...   0   0   0]
 [  4  42   3 ...   0   0   0]]
[8]


In [177]:
output = vectorizer([["I tried running this line of code, but I'm receiving a null pointer exception"]])
output.numpy()

array([[   4,  118,  180,   11,  161,    9,   39,   15,   26, 1842,    5,
         381,  842,  297,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [201]:
voc = vectorizer.get_vocabulary()
v = [voci.decode("utf-8") for voci in voc]
word_index = dict(zip(v, range(len(v))))

In [203]:
#test = [bytes("this" "utf-8"), bytes("line", "utf-8"), bytes("of", "utf-8"), bytes("code", "utf-8")]
test = ["this", "line", "of", "code"]
print(test)
print(v[9])
print(len(word_index))
widx=0
for w, i in word_index.items():
    print(str(w) + ":" + str(i))
    widx+=1
    if widx==20:
        break
print([word_index[w] for w in test])

['this', 'line', 'of', 'code']
this
19999
the:0
to:1
i:2
a:3
is:4
and:5
in:6
of:7
it:8
this:9
that:10
for:11
have:12
but:13
my:14
on:15
with:16
be:17
can:18
not:19
[9, 159, 7, 37]


In [207]:
num_tokens = len(voc) + 2
embedding_dim = 200
hits = 0
misses = 0
miss_word = []

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = word_vect.get_vector(word)
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    except:
        miss_word.append(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
print(str(miss_word))
print("--------------")
print(word_vect.get_vector("c#"))

Converted 18811 words (1188 misses)
['the', 'to', 'i', 'a', 'is', 'and', 'in', 'of', 'it', 'this', 'that', 'for', 'have', 'but', 'my', 'on', 'with', 'be', 'can', 'not', 'an', 'if', 'as', 'do', 'im', 'how', 'so', 'or', 'from', 'there', 'when', 'am', 'are', 'what', 'any', 'would', 'which', 'some', 'all', 'you', 'at', 'me', 'by', 'its', 'has', 'then', 'does', 'only', 'should', 'was', 'other', 'out', 'into', 'dont', 'we', 'same', 'no', 'up', 'about', 'here', 'where', 'could', 'each', 'more', 'they', 'them', 'doesnt', 'these', 'cant', 'because', 'after', 'why', 'been', 'doing', 'than', 'very', 'through', 'being', 'having', 'such', 'before', 'both', 'your', 'their', 'below', 'while', 'between', 'over', 'above', 'our', 'cannot', 'those', 'most', 'had', 'own', 'too', 'eg', 'few', 'were', 'heres', 'again', 'down', 'isnt', 'did', 'once', 'whats', 'thats', 'under', 'theres', 'didnt', 'let', 'off', 'until', 'itself', 'lets', 'wont', 'who', 'during', 'he', 'havent', 'myself', 'against', 'couldnt', 

In [208]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [209]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(mostCommonTags))(x) #change to all tags
preds = layers.Softmax(axis=-1)(preds)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 200)         4000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         128128    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048 

In [210]:
from sklearn.utils import class_weight

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

weights = {}

for i in range(10):
    weights[i] = (1 / tagToFrequencyList[i][1])

model.fit(train_X, train_y, batch_size=128, epochs=20, validation_data=(test_X, test_y), class_weight=weights)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 75000 samples, validate on 25000 samples
Epoch 1/20
75000/75000 [==============================] - 21s 274us/sample - loss: 2.2414e-06 - accuracy: 0.9492 - val_loss: 2.1941e-06 - val_accuracy: 0.9494
Epoch 2/20
75000/75000 [==============================] - 15s 194us/sample - loss: 2.2045e-06 - accuracy: 0.9495 - val_loss: 2.1827e-06 - val_accuracy: 0.9494
Epoch 3/20
75000/75000 [==============================] - 15s 195us/sample - loss: 2.1935e-06 - accuracy: 0.9495 - val_loss: 2.1779e-06 - val_accuracy: 0.9494
Epoch 4/20
75000/75000 [==============================] - 15s 194us/sample - loss: 2.1867e-06 - accuracy: 0.9495 - val_loss: 2.1733e-06 - val_accuracy: 0.9494
Epoch 5/20
75000/75000 [==============================] - 15s 194us/sample - loss: 2.1771e-06 - accuracy: 0.9495 - val_loss: 2.1639e-06 - val_accuracy: 0.9494
Epoch 6/20
75000/75000 [==============================] - 14s 193us/sample - loss: 2.1640e-06 - accuracy:

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    ["can someone explain me how system.out.println works"]
#     ["Dropped my iphone again :("]
)

for i, prob in np.ndenumerate(probabilities):
    print('{:<16}  {:<16}'.format(tagIndexToTag[tagToFrequencyList[i[1]][0]], truncate(prob, 3)))
print(f"\nMost likely tag: {tagIndexToTag[tagToFrequencyList[np.argmax(probabilities)][0]]}")